In [23]:
import json

from enum import Enum

class Mark(Enum):
    GREEN = 0
    ORANGE = 5
    RED = 10

In [24]:
class Interval:
    def __init__(self, left: float, right: float):
        if not isinstance(left, float) or not isinstance(right, float):
            raise TypeError("Левая и правая границы должны быть вещественными числами.")
        if left > right:
            raise ValueError("Левая граница должна быть меньше или равна правой границе.")
        self.left = float(left)
        self.right = float(right)

    # Проверка пересечения двух интервалов
    def intersects(self, other):
        return self.left <= other.right and other.left <= self.right

    # Проверка включения одного интервала в другой
    def contains_interval(self, other):
        return self.left <= other.left and self.right >= other.right

    # Проверка совпадения двух интервалов
    def __eq__(self, other):
        return self.left == other.left and self.right == other.right

    # Проверка принадлежности числа интервалу
    def contains_value(self, value):
        return self.left <= value <= self.right

    # Вычисление длины интервала
    def length(self):
        return self.right - self.left

    # Получение нижней границы интервала
    def get_left(self):
        return self.left

    # Получение верхней границы интервала
    def get_right(self):
        return self.right

In [25]:
with open("response.json", "r", encoding="utf-8") as file:
        json_file = json.loads(file.read())

In [26]:
def find_meta_value(data, meta):
  """
  Находит первый объект с заданным значением ключа "meta".

  Args:
    data: Данные в формате словаря или списка.
    meta: Искомое значение ключа "meta".

  Returns:
    Словарь с найденным объектом или None, если объект не найден.
  """
  if isinstance(data, dict):
    if 'meta' in data and data['meta'] == meta:
      return data
    for value in data.values():
      result = find_meta_value(value, meta)
      if result:
        return result
  elif isinstance(data, list):
    for item in data:
      result = find_meta_value(item, meta)
      if result:
        return result
  return None

In [27]:

def find_name_value(data, name):
    """
     Function to find an object with a given key "name" and its value.

     Args:
     data: Data in dictionary format.
     name: The searched value of the key "name".

     Returns:
     A dictionary with the found object or None if the object is not found.
    """

    if 'name' in data and data['name'] == name:
        return data

    for item in data.values():
        if isinstance(item, dict):
            result = find_name_value(item, name)
            if result:
                return result
        elif isinstance(item, list):
            for sub_item in item:
                result = find_name_value(sub_item, name)
                if result:
                    return result

    return None

In [38]:
def find_nested_element(data, key1, value1, key2, value2):
    def find_first_pair(data, key1, value1):
        if isinstance(data, dict):
            if key1 in data and data[key1] == value1:
                return data
            for key, value in data.items():
                if isinstance(value, dict):
                    result = find_first_pair(value, key1, value1)
                    if result is not None:
                        return result
                elif isinstance(value, list):
                    for item in value:
                        if isinstance(item, dict):
                            result = find_first_pair(item, key1, value1)
                            if result is not None:
                                return result
        elif isinstance(data, list):
            for item in data:
                result = find_first_pair(item, key1, value1)
                if result is not None:
                    return result
        return None

    first_pair_dict = find_first_pair(data, key1, value1)

    if first_pair_dict:
        return find_first_pair(first_pair_dict, key2, value2)

    return None

In [28]:
result_1 = find_name_value(json_file,"Сталь марки РС категории А")

In [29]:
result_1 = find_name_value(result_1,"Элементный состав")

In [30]:
find_name_value(json_file,"Ст3сп")

{'id': 381405980799364,
 'name': 'Ст3сп',
 'type': 'НЕТЕРМИНАЛ',
 'meta': 'Материал',
 'successors': [{'id': 381405980799368,
   'name': 'Элементный состав',
   'type': 'НЕТЕРМИНАЛ',
   'meta': 'Элементный состав',
   'successors': [{'id': 381405980799372,
     'name': 1,
     'type': 'НЕТЕРМИНАЛ',
     'meta': 'Компонент',
     'successors': [{'id': 381405980799376,
       'name': 'Fe',
       'type': 'НЕТЕРМИНАЛ',
       'meta': 'Химический элемент',
       'original': 'vadim@dvo.ru / Мой Фонд / Лазерное аддитивное производство / База химических элементов$/Железо/Fe;',
       'successors': [{'id': 381405980799382,
         'value': '%',
         'type': 'ТЕРМИНАЛ-ЗНАЧЕНИЕ',
         'valtype': 'STRING',
         'meta': '%'},
        {'id': 381405980799386,
         'value': 97.0,
         'type': 'ТЕРМИНАЛ-ЗНАЧЕНИЕ',
         'valtype': 'REAL',
         'meta': 'Числовое значение'}]}]},
    {'id': 381405980799396,
     'name': 2,
     'type': 'НЕТЕРМИНАЛ',
     'meta': 'Компонент',


In [45]:
def osnov_compare(json1,json2):
    osnov_1 = find_meta_value(json1,"Основа")
    osnov_2 = find_meta_value(json2,"Основа")
   
    values_1 = [item['value'] for item in osnov_1['successors']]
    values_2 = [item['value'] for item in osnov_2['successors']]

    if values_1 == values_2 :


        if json1 is None and  json2 is not None:
             return Mark.RED 

        elif json1 is None and  json2 is  None:
            return Mark.GREEN

        else:
            for component in json1["successors"][:2]:
                
                chim_element_1 = find_meta_value(component,"Химический элемент")
                result  = find_nested_element(component, "meta" ,component["meta"], "meta","Химический элемент" )
                
                print(result)






            
    else: 
        return Mark.RED
        


In [46]:
osnov_compare(result_1,result_1)

{'id': 381405980798192, 'name': 'Fe', 'type': 'НЕТЕРМИНАЛ', 'meta': 'Химический элемент', 'original': 'vadim@dvo.ru / Мой Фонд / Лазерное аддитивное производство / База химических элементов$/Железо/Fe;', 'successors': [{'id': 381405980798198, 'value': '%', 'type': 'ТЕРМИНАЛ-ЗНАЧЕНИЕ', 'valtype': 'STRING', 'meta': '%'}]}
{'id': 381405980787756, 'name': 'C', 'type': 'НЕТЕРМИНАЛ', 'meta': 'Химический элемент', 'original': 'vadim@dvo.ru / Мой Фонд / Лазерное аддитивное производство / База химических элементов$/Углерод/C;', 'successors': [{'id': 381405980793246, 'value': '%', 'type': 'ТЕРМИНАЛ-ЗНАЧЕНИЕ', 'valtype': 'STRING', 'meta': '%'}, {'id': 381405980793248, 'name': 'Не более', 'type': 'НЕТЕРМИНАЛ', 'meta': 'Не более', 'successors': [{'id': 381405980793254, 'value': '≤', 'type': 'ТЕРМИНАЛ-ЗНАЧЕНИЕ', 'valtype': 'STRING', 'meta': '≤'}, {'id': 381405980795894, 'value': 0.21, 'type': 'ТЕРМИНАЛ-ЗНАЧЕНИЕ', 'valtype': 'REAL', 'meta': 'Числовое значение'}]}]}
